# 업종 인식률 개선 분석

## 목적
- 21개 하위 그룹별 테스트 데이터를 활용하여 업종 인식 일치율 평가
- 업종 인식 실패 케이스 분석 및 `industries.yaml` 개선 방안 도출

## 분석 과정
1. 테스트 데이터 정의 (21개 그룹 × 10개 예문 = 210개)
2. PromptTemplateManager를 통한 업종 인식 테스트
3. 업종 인식 일치율 평가 및 시각화
4. API를 활용한 실패 원인 분석 및 개선안 보고서 생성

## 1. 환경 설정 및 모듈 로드

In [1]:
import os
import sys
import json
from typing import Dict, List
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 프로젝트 루트 경로 설정
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# 환경변수 로드
load_dotenv()

# 업종 인식용 모듈 import
from src.generation.image_generation.prompt.prompt_manager import PromptTemplateManager

print(f"프로젝트 경로: {PROJECT_ROOT}")
print("모듈 로드 완료!")

프로젝트 경로: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz
모듈 로드 완료!


In [2]:
# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 프롬프트 매니저 초기화
prompt_manager = PromptTemplateManager()

print("PromptTemplateManager 초기화 완료!")

PromptTemplateManager 초기화 완료!


In [3]:
from src.generation.image_generation.prompt import industry_config

# 1. YAML 파일 수정 후 강제로 다시 로드
# industry_config()

# 2. 로드된 데이터가 최신인지 확인 (예: metadata 버전 출력)
current_version = industry_config.config.get('metadata', {}).get('version', 'unknown')
print(f"현재 적용된 YAML 버전: {current_version}")

현재 적용된 YAML 버전: 3.2.0


## 2. 테스트 데이터 정의

### 21개 하위 그룹별 10개의 한글 예시 문장

In [4]:
# ============================================
# S등급: 감성 및 미각 자극 그룹 (6개 그룹, 53개 업종)
# 각 업종별 2개의 광고 생성용 예시 문장
# ============================================

# S-1: 고온 조리 및 육즙 강조 (17개 업종)
S1_HOT_COOKING = [
    # 돼지고기 구이/찜
    {"업종": "돼지고기 구이/찜", "group": "S-1", "input": "숯불에 구운 삼겹살, 육즙이 뚝뚝"},
    {"업종": "돼지고기 구이/찜", "group": "S-1", "input": "껍데기 맛집 홍보, 바삭한 돼지 껍데기"},
    # 소고기 구이/찜
    {"업종": "소고기 구이/찜", "group": "S-1", "input": "한우 등심 구이, 마블링이 살아있는"},
    {"업종": "소고기 구이/찜", "group": "S-1", "input": "갈비찜 전문점, 부드러운 갈비살"},
    # 곱창 전골/구이
    {"업종": "곱창 전골/구이", "group": "S-1", "input": "곱창 구이 맛집, 불판 위 지글지글"},
    {"업종": "곱창 전골/구이", "group": "S-1", "input": "곱창 전골, 얼큰한 국물이 일품"},
    # 닭/오리고기 구이/찜
    {"업종": "닭/오리고기 구이/찜", "group": "S-1", "input": "오리 훈제 전문, 촉촉한 오리 고기"},
    {"업종": "닭/오리고기 구이/찜", "group": "S-1", "input": "찜닭 맛집, 매콤달콤 양념 찜닭"},
    # 치킨 전문점
    {"업종": "치킨 전문점", "group": "S-1", "input": "치킨 전문점, 갓 튀긴 바삭한 치킨"},
    {"업종": "치킨 전문점", "group": "S-1", "input": "양념치킨 배달, 매콤달콤 양념 가득"},
    # 피자 전문점
    {"업종": "피자 전문점", "group": "S-1", "input": "피자 가게 신메뉴, 치즈가 늘어나는"},
    {"업종": "피자 전문점", "group": "S-1", "input": "화덕 피자 전문, 바삭한 도우와 토핑"},
    # 버거 전문점
    {"업종": "버거 전문점", "group": "S-1", "input": "수제버거 전문, 육즙 가득한 패티"},
    {"업종": "버거 전문점", "group": "S-1", "input": "버거 세트 홍보, 감자튀김과 함께"},
    # 마라탕/훠궈 전문점
    {"업종": "마라탕/훠궈 전문점", "group": "S-1", "input": "마라탕 전문점, 얼얼하게 매운 국물"},
    {"업종": "마라탕/훠궈 전문점", "group": "S-1", "input": "훠궈 맛집, 진한 육수와 다양한 재료"},
    # 파스타/스테이크 전문점
    {"업종": "파스타/스테이크 전문점", "group": "S-1", "input": "스테이크 레스토랑, 미디엄 레어 안심"},
    {"업종": "파스타/스테이크 전문점", "group": "S-1", "input": "파스타 전문점, 크림파스타 런치 세트"},
    # 경양식
    {"업종": "경양식", "group": "S-1", "input": "경양식 돈까스, 바삭한 수제 돈까스"},
    {"업종": "경양식", "group": "S-1", "input": "함박 스테이크, 푸짐한 런치 정식"},
    # 패밀리레스토랑
    {"업종": "패밀리레스토랑", "group": "S-1", "input": "패밀리 레스토랑, 온 가족 외식 코스"},
    {"업종": "패밀리레스토랑", "group": "S-1", "input": "뷔페식 레스토랑, 다양한 메뉴 한 곳에"},
    # 일식 카레/돈가스/덮밥
    {"업종": "일식 카레/돈가스/덮밥", "group": "S-1", "input": "일본식 카레 전문, 진한 카레 소스"},
    {"업종": "일식 카레/돈가스/덮밥", "group": "S-1", "input": "규동 전문점, 푸짐한 소고기 덮밥"},
    # 뷔페
    {"업종": "뷔페", "group": "S-1", "input": "호텔 뷔페, 프리미엄 코스 무한리필"},
    {"업종": "뷔페", "group": "S-1", "input": "해산물 뷔페, 신선한 해산물 한상"},
    # 구내식당
    {"업종": "구내식당", "group": "S-1", "input": "구내식당 운영, 직원 복지 식사"},
    {"업종": "구내식당", "group": "S-1", "input": "사내 식당, 건강한 한끼 제공"},
    # 중국집
    {"업종": "중국집", "group": "S-1", "input": "중국집 짜장면, 윤기 나는 면발"},
    {"업종": "중국집", "group": "S-1", "input": "짬뽕 전문점, 얼큰한 해물 짬뽕"},
    # 일식 면 요리
    {"업종": "일식 면 요리", "group": "S-1", "input": "라멘 전문점, 진한 돈코츠 라멘"},
    {"업종": "일식 면 요리", "group": "S-1", "input": "우동 맛집, 쫄깃한 면발의 정석"},
    # 베트남식 전문점
    {"업종": "베트남식 전문점", "group": "S-1", "input": "쌀국수 전문점, 허브와 숙주 가득"},
    {"업종": "베트남식 전문점", "group": "S-1", "input": "반미 전문점, 바삭한 바게트 샌드위치"},
]

# S-2: 신선도 및 색감 강조 (8개 업종)
S2_FRESHNESS = [
    # 횟집
    {"업종": "횟집", "group": "S-2", "input": "횟집 홍보, 싱싱한 모둠회 한상"},
    {"업종": "횟집", "group": "S-2", "input": "자연산 회 전문, 신선도가 다른"},
    # 해산물 구이/찜
    {"업종": "해산물 구이/찜", "group": "S-2", "input": "해물찜 전문, 게와 새우가 가득"},
    {"업종": "해산물 구이/찜", "group": "S-2", "input": "조개구이 맛집, 싱싱한 조개 한판"},
    # 복 요리 전문점
    {"업종": "복 요리 전문점", "group": "S-2", "input": "복어 요리 전문, 복어회와 복어탕"},
    {"업종": "복 요리 전문점", "group": "S-2", "input": "복국 맛집, 담백한 복국 한 그릇"},
    # 일식 회/초밥
    {"업종": "일식 회/초밥", "group": "S-2", "input": "초밥 전문점, 장인의 오마카세"},
    {"업종": "일식 회/초밥", "group": "S-2", "input": "사시미 전문, 신선한 생선회"},
    # 수산물 소매업
    {"업종": "수산물 소매업", "group": "S-2", "input": "수산시장, 산지직송 싱싱한 해산물"},
    {"업종": "수산물 소매업", "group": "S-2", "input": "수산물 직판장, 활어 도소매"},
    # 건어물/젓갈 소매업
    {"업종": "건어물/젓갈 소매업", "group": "S-2", "input": "젓갈 전문점, 전통 방식 명란젓"},
    {"업종": "건어물/젓갈 소매업", "group": "S-2", "input": "건어물 도매, 마른 오징어 대량 판매"},
    # 정육점
    {"업종": "정육점", "group": "S-2", "input": "정육점 홍보, 마블링 좋은 한우"},
    {"업종": "정육점", "group": "S-2", "input": "정육 직판장, 신선한 고기 배달"},
    # 채소/과일 소매업
    {"업종": "채소/과일 소매업", "group": "S-2", "input": "과일 전문점, 제철 딸기 선물세트"},
    {"업종": "채소/과일 소매업", "group": "S-2", "input": "채소 가게, 유기농 신선 채소"},
]

# S-3: 감성 및 부드러운 질감 (11개 업종)
S3_EMOTIONAL = [
    # 카페
    {"업종": "카페", "group": "S-3", "input": "카페 신메뉴, 딸기 라떼와 크루아상"},
    {"업종": "카페", "group": "S-3", "input": "디저트 카페, 수제 케이크 전문"},
    # 빵/도넛 전문점
    {"업종": "빵/도넛 전문점", "group": "S-3", "input": "베이커리, 갓 구운 소금빵"},
    {"업종": "빵/도넛 전문점", "group": "S-3", "input": "도넛 전문점, 달콤한 수제 도넛"},
    # 아이스크림/빙수 전문점
    {"업종": "아이스크림/빙수 전문점", "group": "S-3", "input": "젤라또 전문, 파스텔 톤 아이스크림"},
    {"업종": "아이스크림/빙수 전문점", "group": "S-3", "input": "빙수 카페, 여름 시즌 팥빙수"},
    # 아이스크림 할인점
    {"업종": "아이스크림 할인점", "group": "S-3", "input": "아이스크림 할인점, 인기 제품 반값"},
    {"업종": "아이스크림 할인점", "group": "S-3", "input": "아이스크림 도매, 대량 구매 할인"},
    # 떡/한과 전문점
    {"업종": "떡/한과 전문점", "group": "S-3", "input": "떡 카페, 화과자와 전통차 세트"},
    {"업종": "떡/한과 전문점", "group": "S-3", "input": "한과 전문점, 명절 선물세트"},
    # 토스트/샌드위치/샐러드
    {"업종": "토스트/샌드위치/샐러드", "group": "S-3", "input": "샌드위치 전문점, 건강한 아침 메뉴"},
    {"업종": "토스트/샌드위치/샐러드", "group": "S-3", "input": "샐러드 카페, 다이어트 식단 전문"},
    # 슈퍼마켓
    {"업종": "슈퍼마켓", "group": "S-3", "input": "슈퍼마켓, 신선식품 할인 행사"},
    {"업종": "슈퍼마켓", "group": "S-3", "input": "동네 마트, 생필품 최저가"},
    # 편의점
    {"업종": "편의점", "group": "S-3", "input": "편의점, 간편식 도시락 신제품"},
    {"업종": "편의점", "group": "S-3", "input": "24시 편의점, 야간 배달 서비스"},
    # 생수/음료 소매업
    {"업종": "생수/음료 소매업", "group": "S-3", "input": "생수 배달, 정기 배송 할인"},
    {"업종": "생수/음료 소매업", "group": "S-3", "input": "음료 도매, 여름 음료 대량 구매"},
    # 우유 소매업
    {"업종": "우유 소매업", "group": "S-3", "input": "우유 배달, 신선한 목장 우유"},
    {"업종": "우유 소매업", "group": "S-3", "input": "유제품 전문, 치즈와 요거트"},
    # 그 외 기타 간이 음식점
    {"업종": "그 외 기타 간이 음식점", "group": "S-3", "input": "간이 음식점, 빠르고 맛있는 한끼"},
    {"업종": "그 외 기타 간이 음식점", "group": "S-3", "input": "푸드트럭, 이동식 음식 판매"},
]

# S-4: 정갈한 한식과 상차림 (9개 업종)
S4_KOREAN_HOMESTYLE = [
    # 백반/한정식
    {"업종": "백반/한정식", "group": "S-4", "input": "한정식, 정갈한 반찬 상차림"},
    {"업종": "백반/한정식", "group": "S-4", "input": "백반집, 오늘의 집밥 정식"},
    # 국/탕/찌개류
    {"업종": "국/탕/찌개류", "group": "S-4", "input": "국밥집, 뜨끈한 돼지국밥"},
    {"업종": "국/탕/찌개류", "group": "S-4", "input": "찌개 전문, 된장찌개 백반"},
    # 족발/보쌈
    {"업종": "족발/보쌈", "group": "S-4", "input": "족발 전문점, 쫄깃한 족발"},
    {"업종": "족발/보쌈", "group": "S-4", "input": "보쌈 맛집, 부드러운 수육"},
    # 전/부침개
    {"업종": "전/부침개", "group": "S-4", "input": "전 전문점, 해물파전과 막걸리"},
    {"업종": "전/부침개", "group": "S-4", "input": "부침개 맛집, 바삭한 녹두전"},
    # 국수/칼국수
    {"업종": "국수/칼국수", "group": "S-4", "input": "칼국수집, 손칼국수와 왕만두"},
    {"업종": "국수/칼국수", "group": "S-4", "input": "잔치국수 전문, 멸치 육수 국수"},
    # 냉면/밀면
    {"업종": "냉면/밀면", "group": "S-4", "input": "냉면 전문점, 시원한 물냉면"},
    {"업종": "냉면/밀면", "group": "S-4", "input": "밀면 맛집, 부산식 밀면"},
    # 기타 한식 음식점
    {"업종": "기타 한식 음식점", "group": "S-4", "input": "한식당, 엄마 손맛 집밥"},
    {"업종": "기타 한식 음식점", "group": "S-4", "input": "가정식 백반, 건강한 한끼"},
    # 김밥/만두/분식
    {"업종": "김밥/만두/분식", "group": "S-4", "input": "김밥 전문점, 다양한 김밥"},
    {"업종": "김밥/만두/분식", "group": "S-4", "input": "분식집, 떡볶이와 순대"},
    # 반찬/식료품 소매업
    {"업종": "반찬/식료품 소매업", "group": "S-4", "input": "반찬 가게, 정성 가득 수제 반찬"},
    {"업종": "반찬/식료품 소매업", "group": "S-4", "input": "식료품점, 건강한 먹거리"},
]

# S-5: 글로벌 다채로움 (4개 업종)
S5_GLOBAL_VARIETY = [
    # 기타 일식 음식점
    {"업종": "기타 일식 음식점", "group": "S-5", "input": "일식당, 오코노미야키와 타코야끼"},
    {"업종": "기타 일식 음식점", "group": "S-5", "input": "이자카야, 일본 가정식 요리"},
    # 기타 서양식 음식점
    {"업종": "기타 서양식 음식점", "group": "S-5", "input": "양식당, 리조또와 뇨끼 세트"},
    {"업종": "기타 서양식 음식점", "group": "S-5", "input": "유러피안 레스토랑, 코스 요리"},
    # 기타 동남아식 전문점
    {"업종": "기타 동남아식 전문점", "group": "S-5", "input": "태국 음식점, 팟타이와 쏨땀"},
    {"업종": "기타 동남아식 전문점", "group": "S-5", "input": "동남아 전문, 그린커리 런치"},
    # 분류 안된 외국식 음식점
    {"업종": "분류 안된 외국식 음식점", "group": "S-5", "input": "퓨전 레스토랑, 다국적 요리"},
    {"업종": "분류 안된 외국식 음식점", "group": "S-5", "input": "외국 음식 전문, 세계 맛 체험"},
]

# S-6: 화려한 무드와 야간 감성 (4개 업종)
S6_NIGHTLIFE = [
    # 생맥주 전문점
    {"업종": "생맥주 전문점", "group": "S-6", "input": "생맥주 전문점, 시원한 맥주 한잔"},
    {"업종": "생맥주 전문점", "group": "S-6", "input": "크래프트 비어펍, 수제 맥주 전문"},
    # 요리 주점
    {"업종": "요리 주점", "group": "S-6", "input": "요리주점, 맛있는 안주와 함께"},
    {"업종": "요리 주점", "group": "S-6", "input": "이자카야, 일본식 선술집"},
    # 일반 유흥 주점
    {"업종": "일반 유흥 주점", "group": "S-6", "input": "칵테일 바, 분위기 있는 한잔"},
    {"업종": "일반 유흥 주점", "group": "S-6", "input": "와인바, 와인과 함께하는 밤"},
    # 무도 유흥 주점
    {"업종": "무도 유흥 주점", "group": "S-6", "input": "클럽, 화려한 조명과 음악"},
    {"업종": "무도 유흥 주점", "group": "S-6", "input": "나이트클럽, 흥겨운 밤의 파티"},
]

# S등급 전체 테스트 케이스
S_GRADE_TEST_CASES = S1_HOT_COOKING + S2_FRESHNESS + S3_EMOTIONAL + S4_KOREAN_HOMESTYLE + S5_GLOBAL_VARIETY + S6_NIGHTLIFE

print(f"S등급 테스트 케이스 수: {len(S_GRADE_TEST_CASES)}개 (업종별 2개씩)")

S등급 테스트 케이스 수: 106개 (업종별 2개씩)


In [5]:
# ============================================
# A등급: 비주얼 변화 및 자기관리 그룹 (4개 그룹, 41개 업종)
# 각 업종별 2개의 광고 생성용 예시 문장
# ============================================

# A-1: 인물 중심 뷰티 및 에스테틱 (6개 업종)
A1_BEAUTY = [
    # 미용실
    {"업종": "미용실", "group": "A-1", "input": "미용실, 트렌디한 헤어 스타일링"},
    {"업종": "미용실", "group": "A-1", "input": "헤어샵, 봄 시즌 염색 이벤트"},
    # 피부 관리실
    {"업종": "피부 관리실", "group": "A-1", "input": "피부관리실, 맑고 깨끗한 피부"},
    {"업종": "피부 관리실", "group": "A-1", "input": "에스테틱, 페이셜 케어 전문"},
    # 피부/비뇨기과 의원
    {"업종": "피부/비뇨기과 의원", "group": "A-1", "input": "피부과, 레이저 시술 전후"},
    {"업종": "피부/비뇨기과 의원", "group": "A-1", "input": "피부과 의원, 잡티 제거 상담"},
    # 성형외과 의원
    {"업종": "성형외과 의원", "group": "A-1", "input": "성형외과, 자연스러운 코 성형"},
    {"업종": "성형외과 의원", "group": "A-1", "input": "성형외과 상담, 눈 성형 전문"},
    # 네일숍
    {"업종": "네일숍", "group": "A-1", "input": "네일샵, 봄 꽃 네일 아트"},
    {"업종": "네일숍", "group": "A-1", "input": "네일 전문, 젤네일 디자인"},
    # 화장품 소매업
    {"업종": "화장품 소매업", "group": "A-1", "input": "화장품 매장, 신제품 립스틱"},
    {"업종": "화장품 소매업", "group": "A-1", "input": "코스메틱샵, 스킨케어 상담"},
]

# A-2: 신체 변화 및 웰니스 (6개 업종)
A2_WELLNESS = [
    # 헬스장
    {"업종": "헬스장", "group": "A-2", "input": "헬스장 PT, 1:1 개인 트레이닝"},
    {"업종": "헬스장", "group": "A-2", "input": "피트니스 센터, 신규 회원 모집"},
    # 요가/필라테스 학원
    {"업종": "요가/필라테스 학원", "group": "A-2", "input": "요가 스튜디오, 아침 명상 클래스"},
    {"업종": "요가/필라테스 학원", "group": "A-2", "input": "필라테스 센터, 체형 교정"},
    # 태권도/무술학원
    {"업종": "태권도/무술학원", "group": "A-2", "input": "태권도 도장, 어린이 태권도 수업"},
    {"업종": "태권도/무술학원", "group": "A-2", "input": "무술학원, 주짓수와 복싱 레슨"},
    # 종합 스포츠시설
    {"업종": "종합 스포츠시설", "group": "A-2", "input": "종합 스포츠시설, 다양한 운동"},
    {"업종": "종합 스포츠시설", "group": "A-2", "input": "스포츠 센터, 회원권 할인"},
    # 체형/비만 관리
    {"업종": "체형/비만 관리", "group": "A-2", "input": "다이어트 클리닉, 체중 감량 성공"},
    {"업종": "체형/비만 관리", "group": "A-2", "input": "체형관리실, 비만 관리 프로그램"},
    # 마사지/안마
    {"업종": "마사지/안마", "group": "A-2", "input": "마사지샵, 타이 마사지 전신 케어"},
    {"업종": "마사지/안마", "group": "A-2", "input": "안마원, 스포츠 마사지 전문"},
]

# A-3: 패션 및 스타일 아이템 (18개 업종)
A3_FASHION = [
    # 남성 의류 소매업
    {"업종": "남성 의류 소매업", "group": "A-3", "input": "남성 의류, 깔끔한 캐주얼 룩"},
    {"업종": "남성 의류 소매업", "group": "A-3", "input": "남성복 매장, 정장 맞춤 제작"},
    # 여성 의류 소매업
    {"업종": "여성 의류 소매업", "group": "A-3", "input": "여성 의류, 봄 신상 원피스"},
    {"업종": "여성 의류 소매업", "group": "A-3", "input": "여성복 매장, 트렌드 패션"},
    # 유아용 의류 소매업
    {"업종": "유아용 의류 소매업", "group": "A-3", "input": "아동복 가게, 귀여운 키즈 패션"},
    {"업종": "유아용 의류 소매업", "group": "A-3", "input": "유아복 전문, 아기 옷 세트"},
    # 한복 소매업
    {"업종": "한복 소매업", "group": "A-3", "input": "한복 전문점, 설날 한복 대여"},
    {"업종": "한복 소매업", "group": "A-3", "input": "한복 맞춤, 돌잔치 한복"},
    # 기타 의류 소매업
    {"업종": "기타 의류 소매업", "group": "A-3", "input": "의류 매장, 다양한 스타일"},
    {"업종": "기타 의류 소매업", "group": "A-3", "input": "패션 아울렛, 할인 행사"},
    # 침구류/커튼 소매업
    {"업종": "침구류/커튼 소매업", "group": "A-3", "input": "침구 전문, 프리미엄 이불 세트"},
    {"업종": "침구류/커튼 소매업", "group": "A-3", "input": "커튼 전문점, 맞춤 커튼 제작"},
    # 가발 소매업
    {"업종": "가발 소매업", "group": "A-3", "input": "가발 전문점, 자연스러운 가발"},
    {"업종": "가발 소매업", "group": "A-3", "input": "위그샵, 패션 가발 판매"},
    # 신발 소매업
    {"업종": "신발 소매업", "group": "A-3", "input": "신발 매장, 트렌디한 스니커즈"},
    {"업종": "신발 소매업", "group": "A-3", "input": "구두 전문점, 수제 구두"},
    # 가방 소매업
    {"업종": "가방 소매업", "group": "A-3", "input": "가방 브랜드, 가죽 토트백"},
    {"업종": "가방 소매업", "group": "A-3", "input": "가방 전문점, 백팩 신상품"},
    # 인테리어 디자인업
    {"업종": "인테리어 디자인업", "group": "A-3", "input": "인테리어 디자인, 모던 거실"},
    {"업종": "인테리어 디자인업", "group": "A-3", "input": "인테리어 시공, 아파트 리모델링"},
    # 제품 디자인업
    {"업종": "제품 디자인업", "group": "A-3", "input": "제품 디자인, 신제품 컨셉"},
    {"업종": "제품 디자인업", "group": "A-3", "input": "산업 디자인, 제품 개발"},
    # 시각 디자인업
    {"업종": "시각 디자인업", "group": "A-3", "input": "시각 디자인, 브랜드 로고"},
    {"업종": "시각 디자인업", "group": "A-3", "input": "그래픽 디자인, 광고 디자인"},
    # 패션/섬유/기타 전문 디자인업
    {"업종": "패션/섬유/기타 전문 디자인업", "group": "A-3", "input": "패션 디자인, 시즌 컬렉션"},
    {"업종": "패션/섬유/기타 전문 디자인업", "group": "A-3", "input": "섬유 디자인, 패턴 개발"},
    # 의류 대여업
    {"업종": "의류 대여업", "group": "A-3", "input": "드레스 대여, 파티 드레스 렌탈"},
    {"업종": "의류 대여업", "group": "A-3", "input": "정장 대여, 면접 정장 렌탈"},
    # 의류/이불 수선업
    {"업종": "의류/이불 수선업", "group": "A-3", "input": "수선집, 옷 수선 전문"},
    {"업종": "의류/이불 수선업", "group": "A-3", "input": "이불 수선, 이불 리폼"},
    # 가죽/가방/신발 수선업
    {"업종": "가죽/가방/신발 수선업", "group": "A-3", "input": "가죽 수선, 명품 가방 수리"},
    {"업종": "가죽/가방/신발 수선업", "group": "A-3", "input": "신발 수선, 구두 굽 교체"},
    # 예식장업
    {"업종": "예식장업", "group": "A-3", "input": "웨딩홀, 프리미엄 결혼식장"},
    {"업종": "예식장업", "group": "A-3", "input": "예식장, 가든 웨딩 전문"},
    # 결혼 상담 서비스업
    {"업종": "결혼 상담 서비스업", "group": "A-3", "input": "결혼 정보회사, 매칭 서비스"},
    {"업종": "결혼 상담 서비스업", "group": "A-3", "input": "웨딩 플래너, 결혼 준비 상담"},
]

# A-4: 섬세한 케어 및 감성 소품 (11개 업종)
A4_DELICATE_CARE = [
    # 꽃집
    {"업종": "꽃집", "group": "A-4", "input": "꽃집, 발렌타인 장미 꽃다발"},
    {"업종": "꽃집", "group": "A-4", "input": "플라워샵, 기념일 꽃배달"},
    # 애완동물/애완용품 소매업
    {"업종": "애완동물/애완용품 소매업", "group": "A-4", "input": "펫샵, 강아지 용품 신상"},
    {"업종": "애완동물/애완용품 소매업", "group": "A-4", "input": "반려동물 용품점, 고양이 간식"},
    # 동물병원
    {"업종": "동물병원", "group": "A-4", "input": "동물병원, 반려견 건강검진"},
    {"업종": "동물병원", "group": "A-4", "input": "수의원, 예방접종 안내"},
    # 가구 소매업
    {"업종": "가구 소매업", "group": "A-4", "input": "가구점, 미니멀 원목 책상"},
    {"업종": "가구 소매업", "group": "A-4", "input": "가구 매장, 소파 할인 행사"},
    # 주방/가정용품 소매업
    {"업종": "주방/가정용품 소매업", "group": "A-4", "input": "주방용품점, 프리미엄 조리도구"},
    {"업종": "주방/가정용품 소매업", "group": "A-4", "input": "리빙 매장, 생활용품 할인"},
    # 건강보조식품 소매업
    {"업종": "건강보조식품 소매업", "group": "A-4", "input": "건강식품, 비타민과 유산균"},
    {"업종": "건강보조식품 소매업", "group": "A-4", "input": "건강보조식품, 프로틴 보충제"},
    # 액세서리/잡화 소매업
    {"업종": "액세서리/잡화 소매업", "group": "A-4", "input": "액세서리 매장, 봄 시즌 주얼리"},
    {"업종": "액세서리/잡화 소매업", "group": "A-4", "input": "잡화점, 패션 소품 판매"},
    # 시계/귀금속 소매업
    {"업종": "시계/귀금속 소매업", "group": "A-4", "input": "시계 매장, 프리미엄 시계"},
    {"업종": "시계/귀금속 소매업", "group": "A-4", "input": "귀금속 전문, 결혼 예물"},
    # 안경렌즈 소매업
    {"업종": "안경렌즈 소매업", "group": "A-4", "input": "안경점, 트렌드 뿔테 안경"},
    {"업종": "안경렌즈 소매업", "group": "A-4", "input": "안경 전문점, 렌즈 맞춤"},
    # 사진기/기타 광학기기 소매업
    {"업종": "사진기/기타 광학기기 소매업", "group": "A-4", "input": "카메라 매장, 미러리스 카메라"},
    {"업종": "사진기/기타 광학기기 소매업", "group": "A-4", "input": "광학기기 전문, 망원경 판매"},
    # 사진촬영업
    {"업종": "사진촬영업", "group": "A-4", "input": "사진관, 가족 기념사진 촬영"},
    {"업종": "사진촬영업", "group": "A-4", "input": "스튜디오, 프로필 촬영 전문"},
]

# A등급 전체 테스트 케이스
A_GRADE_TEST_CASES = A1_BEAUTY + A2_WELLNESS + A3_FASHION + A4_DELICATE_CARE

print(f"A등급 테스트 케이스 수: {len(A_GRADE_TEST_CASES)}개 (업종별 2개씩)")

A등급 테스트 케이스 수: 82개 (업종별 2개씩)


In [6]:
# ============================================
# B등급: 공간 체험 및 교육 그룹 (3개 그룹, 46개 업종)
# 각 업종별 2개의 광고 생성용 예시 문장
# ============================================

# B-1: 학습 및 몰입 (18개 업종)
B1_LEARNING = [
    # 입시·교과학원
    {"업종": "입시·교과학원", "group": "B-1", "input": "입시학원, 명문대 합격 전문"},
    {"업종": "입시·교과학원", "group": "B-1", "input": "교과학원, 소수정예 수학 클래스"},
    # 외국어학원
    {"업종": "외국어학원", "group": "B-1", "input": "외국어학원, 원어민 영어 회화"},
    {"업종": "외국어학원", "group": "B-1", "input": "어학원, 토익 토플 전문반"},
    # 전문자격/고시학원
    {"업종": "전문자격/고시학원", "group": "B-1", "input": "공무원 학원, 합격자 배출"},
    {"업종": "전문자격/고시학원", "group": "B-1", "input": "자격증 학원, 전문 자격 취득"},
    # 사회교육시설
    {"업종": "사회교육시설", "group": "B-1", "input": "평생교육원, 다양한 교육 과정"},
    {"업종": "사회교육시설", "group": "B-1", "input": "문화센터, 취미 클래스 개설"},
    # 직원 훈련기관
    {"업종": "직원 훈련기관", "group": "B-1", "input": "직업훈련원, 취업 교육 전문"},
    {"업종": "직원 훈련기관", "group": "B-1", "input": "기업교육 센터, 직원 역량 강화"},
    # 운전학원
    {"업종": "운전학원", "group": "B-1", "input": "운전학원, 친절한 1:1 교육"},
    {"업종": "운전학원", "group": "B-1", "input": "운전면허 학원, 빠른 합격 보장"},
    # 기타 기술/직업 훈련학원
    {"업종": "기타 기술/직업 훈련학원", "group": "B-1", "input": "기술학원, 용접 자격증 취득"},
    {"업종": "기타 기술/직업 훈련학원", "group": "B-1", "input": "직업훈련 학원, 취업 연계"},
    # 컴퓨터 학원
    {"업종": "컴퓨터 학원", "group": "B-1", "input": "컴퓨터학원, 코딩 교육 전문"},
    {"업종": "컴퓨터 학원", "group": "B-1", "input": "IT학원, 프로그래밍 과정"},
    # 그 외 기타 교육기관
    {"업종": "그 외 기타 교육기관", "group": "B-1", "input": "교육기관, 맞춤형 교육 과정"},
    {"업종": "그 외 기타 교육기관", "group": "B-1", "input": "학습센터, 개인 맞춤 지도"},
    # 교육컨설팅업
    {"업종": "교육컨설팅업", "group": "B-1", "input": "유학원, 미국 대학 입학 컨설팅"},
    {"업종": "교육컨설팅업", "group": "B-1", "input": "교육컨설팅, 진로 상담"},
    # 기타 교육지원 서비스업
    {"업종": "기타 교육지원 서비스업", "group": "B-1", "input": "교육지원 서비스, 학습 자료 제공"},
    {"업종": "기타 교육지원 서비스업", "group": "B-1", "input": "에듀테크, 온라인 학습 플랫폼"},
    # 독서실/스터디 카페
    {"업종": "독서실/스터디 카페", "group": "B-1", "input": "스터디카페, 조용한 학습 공간"},
    {"업종": "독서실/스터디 카페", "group": "B-1", "input": "독서실, 24시간 개인 부스"},
    # 청소년 수련시설
    {"업종": "청소년 수련시설", "group": "B-1", "input": "청소년 수련원, 체험 활동"},
    {"업종": "청소년 수련시설", "group": "B-1", "input": "수련시설, 단체 캠프"},
    # 만화방
    {"업종": "만화방", "group": "B-1", "input": "만화카페, 신간 만화 가득"},
    {"업종": "만화방", "group": "B-1", "input": "만화방, 아늑한 독서 공간"},
    # 음악학원
    {"업종": "음악학원", "group": "B-1", "input": "음악학원, 피아노 레슨 전문"},
    {"업종": "음악학원", "group": "B-1", "input": "음악 교습소, 기타 배우기"},
    # 미술학원
    {"업종": "미술학원", "group": "B-1", "input": "미술학원, 수채화 클래스"},
    {"업종": "미술학원", "group": "B-1", "input": "그림 학원, 입시 미술 전문"},
    # 기타 예술/스포츠 교육기관
    {"업종": "기타 예술/스포츠 교육기관", "group": "B-1", "input": "무용학원, 발레 레슨"},
    {"업종": "기타 예술/스포츠 교육기관", "group": "B-1", "input": "스포츠 아카데미, 축구 교실"},
    # 레크리에이션 교육기관
    {"업종": "레크리에이션 교육기관", "group": "B-1", "input": "레크리에이션 교육, 놀이 지도"},
    {"업종": "레크리에이션 교육기관", "group": "B-1", "input": "체험학습 센터, 창의 활동"},
]

# B-2: 여행 및 숙박 (9개 업종)
B2_LEISURE = [
    # 호텔/리조트
    {"업종": "호텔/리조트", "group": "B-2", "input": "호텔, 오션뷰 스위트룸 패키지"},
    {"업종": "호텔/리조트", "group": "B-2", "input": "리조트, 가족 휴가 추천"},
    # 여관/모텔
    {"업종": "여관/모텔", "group": "B-2", "input": "모텔, 깔끔한 비즈니스 숙박"},
    {"업종": "여관/모텔", "group": "B-2", "input": "여관, 합리적인 가격 숙소"},
    # 펜션
    {"업종": "펜션", "group": "B-2", "input": "펜션, 가족 단위 바베큐 독채"},
    {"업종": "펜션", "group": "B-2", "input": "풀빌라 펜션, 프라이빗 휴양"},
    # 캠핑/글램핑
    {"업종": "캠핑/글램핑", "group": "B-2", "input": "글램핑장, 자연 속 럭셔리 캠핑"},
    {"업종": "캠핑/글램핑", "group": "B-2", "input": "캠핑장, 오토캠핑 사이트"},
    # 기숙사/고시원
    {"업종": "기숙사/고시원", "group": "B-2", "input": "고시원, 1인실 원룸형 주거"},
    {"업종": "기숙사/고시원", "group": "B-2", "input": "기숙사, 학생 숙소 안내"},
    # 그 외 기타 숙박업
    {"업종": "그 외 기타 숙박업", "group": "B-2", "input": "게스트하우스, 배낭여행 숙소"},
    {"업종": "그 외 기타 숙박업", "group": "B-2", "input": "한옥 숙박, 전통 체험"},
    # 여행사
    {"업종": "여행사", "group": "B-2", "input": "여행사, 제주도 패키지 투어"},
    {"업종": "여행사", "group": "B-2", "input": "투어 여행사, 해외여행 상담"},
    # 기타 여행 보조/예약 서비스업
    {"업종": "기타 여행 보조/예약 서비스업", "group": "B-2", "input": "여행 예약, 항공권 최저가"},
    {"업종": "기타 여행 보조/예약 서비스업", "group": "B-2", "input": "투어 가이드, 현지 안내 서비스"},
    # 목욕탕/사우나
    {"업종": "목욕탕/사우나", "group": "B-2", "input": "찜질방, 힐링 스파 시설"},
    {"업종": "목욕탕/사우나", "group": "B-2", "input": "사우나, 피로 회복 목욕"},
]

# B-3: 스포츠 및 활동 (19개 업종)
B3_SPORTS = [
    # 수영장
    {"업종": "수영장", "group": "B-3", "input": "수영장, 성인 수영 강습"},
    {"업종": "수영장", "group": "B-3", "input": "실내 수영장, 어린이 수영 교실"},
    # 볼링장
    {"업종": "볼링장", "group": "B-3", "input": "볼링장, 단체 볼링 대회"},
    {"업종": "볼링장", "group": "B-3", "input": "볼링 센터, 동호회 대관"},
    # 당구장
    {"업종": "당구장", "group": "B-3", "input": "당구장, 프로 당구 레슨"},
    {"업종": "당구장", "group": "B-3", "input": "포켓볼장, 캐롬 당구"},
    # 골프 연습장
    {"업종": "골프 연습장", "group": "B-3", "input": "골프 연습장, 스크린 골프"},
    {"업종": "골프 연습장", "group": "B-3", "input": "골프 레슨, 초보자 강습"},
    # 테니스장
    {"업종": "테니스장", "group": "B-3", "input": "테니스장, 실내 테니스 코트"},
    {"업종": "테니스장", "group": "B-3", "input": "테니스 클럽, 레슨 프로그램"},
    # 탁구장
    {"업종": "탁구장", "group": "B-3", "input": "탁구장, 동호인 리그전"},
    {"업종": "탁구장", "group": "B-3", "input": "탁구 클럽, 탁구 레슨"},
    # 기타 스포츠시설 운영업
    {"업종": "기타 스포츠시설 운영업", "group": "B-3", "input": "풋살장, 최신 인조잔디 구장"},
    {"업종": "기타 스포츠시설 운영업", "group": "B-3", "input": "배드민턴장, 동호회 대관"},
    # 스쿼시/라켓볼장
    {"업종": "스쿼시/라켓볼장", "group": "B-3", "input": "스쿼시장, 1:1 레슨"},
    {"업종": "스쿼시/라켓볼장", "group": "B-3", "input": "라켓볼장, 코트 대관"},
    # 스포츠/레크리에이션 용품 대여업
    {"업종": "스포츠/레크리에이션 용품 대여업", "group": "B-3", "input": "스포츠 용품 대여, 스키 장비 렌탈"},
    {"업종": "스포츠/레크리에이션 용품 대여업", "group": "B-3", "input": "캠핑용품 대여, 텐트 렌탈"},
    # 노래방
    {"업종": "노래방", "group": "B-3", "input": "노래방, 최신 음향 시스템"},
    {"업종": "노래방", "group": "B-3", "input": "코인 노래방, 혼자서도 OK"},
    # 비디오방
    {"업종": "비디오방", "group": "B-3", "input": "비디오방, 프라이빗 영화 감상"},
    {"업종": "비디오방", "group": "B-3", "input": "DVD방, 커플 영화관"},
    # 전자 게임장
    {"업종": "전자 게임장", "group": "B-3", "input": "오락실, 복고풍 아케이드 게임"},
    {"업종": "전자 게임장", "group": "B-3", "input": "게임 센터, 최신 게임기"},
    # PC방
    {"업종": "PC방", "group": "B-3", "input": "PC방, 고사양 게이밍 PC"},
    {"업종": "PC방", "group": "B-3", "input": "게이밍 카페, 24시간 영업"},
    # 수상/해양 레저업
    {"업종": "수상/해양 레저업", "group": "B-3", "input": "수상 레저, 제트스키 체험"},
    {"업종": "수상/해양 레저업", "group": "B-3", "input": "해양 스포츠, 스쿠버 다이빙"},
    # 낚시터 운영업
    {"업종": "낚시터 운영업", "group": "B-3", "input": "낚시터, 자연 속 힐링 낚시"},
    {"업종": "낚시터 운영업", "group": "B-3", "input": "바다 낚시, 선상 낚시 체험"},
    # 복권 발행/판매업
    {"업종": "복권 발행/판매업", "group": "B-3", "input": "복권 판매점, 로또 명당"},
    {"업종": "복권 발행/판매업", "group": "B-3", "input": "복권방, 스크래치 복권"},
    # 바둑/장기/체스 경기 운영업
    {"업종": "바둑/장기/체스 경기 운영업", "group": "B-3", "input": "바둑 기원, 바둑 레슨"},
    {"업종": "바둑/장기/체스 경기 운영업", "group": "B-3", "input": "장기 도장, 대국 모임"},
    # 기타 오락장
    {"업종": "기타 오락장", "group": "B-3", "input": "방탈출 카페, 테마 방탈출"},
    {"업종": "기타 오락장", "group": "B-3", "input": "VR 체험관, 가상현실 게임"},
    # 기타 오락관련 서비스업
    {"업종": "기타 오락관련 서비스업", "group": "B-3", "input": "파티룸, 생일파티 대관"},
    {"업종": "기타 오락관련 서비스업", "group": "B-3", "input": "보드게임 카페, 다양한 게임"},
]

# ============================================
# C등급: 신뢰 기반 전문 서비스 그룹 (3개 그룹, 57개 업종)
# ============================================

# C-1: 전문 사무 (26개 업종)
C1_PROFESSIONAL = [
    # 부동산 중개/대리업
    {"업종": "부동산 중개/대리업", "group": "C-1", "input": "부동산, 역세권 오피스텔 매물"},
    {"업종": "부동산 중개/대리업", "group": "C-1", "input": "공인중개사, 전월세 매물 안내"},
    # 변호사
    {"업종": "변호사", "group": "C-1", "input": "법률사무소, 기업 법률 자문"},
    {"업종": "변호사", "group": "C-1", "input": "변호사 사무실, 형사 민사 전문"},
    # 변리사
    {"업종": "변리사", "group": "C-1", "input": "특허 사무소, 상표권 등록"},
    {"업종": "변리사", "group": "C-1", "input": "변리사 사무소, 특허 출원"},
    # 법무사
    {"업종": "법무사", "group": "C-1", "input": "법무사 사무소, 등기 업무"},
    {"업종": "법무사", "group": "C-1", "input": "법무사, 소송 서류 작성"},
    # 행정사
    {"업종": "행정사", "group": "C-1", "input": "행정사 사무소, 인허가 대행"},
    {"업종": "행정사", "group": "C-1", "input": "행정사, 민원 서류 작성"},
    # 공인노무사
    {"업종": "공인노무사", "group": "C-1", "input": "노무사 사무실, 근로계약서 검토"},
    {"업종": "공인노무사", "group": "C-1", "input": "노무사, 노동 분쟁 상담"},
    # 기타 법무관련 서비스업
    {"업종": "기타 법무관련 서비스업", "group": "C-1", "input": "법무 서비스, 계약서 검토"},
    {"업종": "기타 법무관련 서비스업", "group": "C-1", "input": "법률 상담, 무료 법률 상담"},
    # 공인회계사
    {"업종": "공인회계사", "group": "C-1", "input": "회계사, 재무 컨설팅"},
    {"업종": "공인회계사", "group": "C-1", "input": "회계법인, 감사 서비스"},
    # 세무사
    {"업종": "세무사", "group": "C-1", "input": "세무사 사무소, 세금 신고 대행"},
    {"업종": "세무사", "group": "C-1", "input": "세무사, 법인세 절세 상담"},
    # 기타 회계 관련 서비스업
    {"업종": "기타 회계 관련 서비스업", "group": "C-1", "input": "회계 서비스, 장부 기장"},
    {"업종": "기타 회계 관련 서비스업", "group": "C-1", "input": "경리 대행, 급여 관리"},
    # 광고 대행업
    {"업종": "광고 대행업", "group": "C-1", "input": "광고 대행사, 마케팅 캠페인"},
    {"업종": "광고 대행업", "group": "C-1", "input": "광고 에이전시, 브랜드 홍보"},
    # 옥외/전시 광고 대행업
    {"업종": "옥외/전시 광고 대행업", "group": "C-1", "input": "옥외 광고, 전광판 광고"},
    {"업종": "옥외/전시 광고 대행업", "group": "C-1", "input": "전시 광고, 박람회 홍보"},
    # 광고 매체 판매업
    {"업종": "광고 매체 판매업", "group": "C-1", "input": "광고 매체, 지면 광고 판매"},
    {"업종": "광고 매체 판매업", "group": "C-1", "input": "미디어렙, 광고 지면 중개"},
    # 광고물 설계/제작업
    {"업종": "광고물 설계/제작업", "group": "C-1", "input": "간판 제작, LED 채널 간판"},
    {"업종": "광고물 설계/제작업", "group": "C-1", "input": "광고물 제작, 현수막 인쇄"},
    # 기타 광고 관련 서비스업
    {"업종": "기타 광고 관련 서비스업", "group": "C-1", "input": "온라인 광고, SNS 마케팅"},
    {"업종": "기타 광고 관련 서비스업", "group": "C-1", "input": "바이럴 마케팅, 인플루언서"},
    # 시장 조사 및 여론 조사업
    {"업종": "시장 조사 및 여론 조사업", "group": "C-1", "input": "시장 조사, 소비자 분석"},
    {"업종": "시장 조사 및 여론 조사업", "group": "C-1", "input": "여론 조사, 설문 조사 전문"},
    # 경영 컨설팅업
    {"업종": "경영 컨설팅업", "group": "C-1", "input": "경영 컨설팅, 기업 성장 전략"},
    {"업종": "경영 컨설팅업", "group": "C-1", "input": "비즈니스 컨설팅, 스타트업 지원"},
    # 기타 사무 지원 서비스업
    {"업종": "기타 사무 지원 서비스업", "group": "C-1", "input": "사무 지원, 비서 서비스"},
    {"업종": "기타 사무 지원 서비스업", "group": "C-1", "input": "사무실 지원, 문서 작성"},
    # 고용 알선업
    {"업종": "고용 알선업", "group": "C-1", "input": "인력 사무소, 일자리 알선"},
    {"업종": "고용 알선업", "group": "C-1", "input": "취업 알선, 맞춤 인력 매칭"},
    # 임시/일용 인력 공급업
    {"업종": "임시/일용 인력 공급업", "group": "C-1", "input": "일용직 알선, 단기 아르바이트"},
    {"업종": "임시/일용 인력 공급업", "group": "C-1", "input": "인력 공급, 현장 인력 파견"},
    # 상용 인력 공급 및 인사관리 서비스업
    {"업종": "상용 인력 공급 및 인사관리 서비스업", "group": "C-1", "input": "인사 관리, HR 아웃소싱"},
    {"업종": "상용 인력 공급 및 인사관리 서비스업", "group": "C-1", "input": "정규직 파견, 인력 헤드헌팅"},
    # 복사업
    {"업종": "복사업", "group": "C-1", "input": "복사집, 문서 인쇄 서비스"},
    {"업종": "복사업", "group": "C-1", "input": "인쇄소, 대량 인쇄 전문"},
    # 전시/컨벤션/행사 대행 서비스업
    {"업종": "전시/컨벤션/행사 대행 서비스업", "group": "C-1", "input": "전시 대행, 박람회 기획"},
    {"업종": "전시/컨벤션/행사 대행 서비스업", "group": "C-1", "input": "행사 대행, 컨퍼런스 운영"},
    # 명함/간판/광고물 제작
    {"업종": "명함/간판/광고물 제작", "group": "C-1", "input": "명함 인쇄, 디자인 명함"},
    {"업종": "명함/간판/광고물 제작", "group": "C-1", "input": "간판 시공, 아크릴 간판"},
    # 번역/통역 서비스업
    {"업종": "번역/통역 서비스업", "group": "C-1", "input": "번역 서비스, 법률 문서 번역"},
    {"업종": "번역/통역 서비스업", "group": "C-1", "input": "통역 서비스, 비즈니스 통역"},
    # 사업/무형 재산권 중개업
    {"업종": "사업/무형 재산권 중개업", "group": "C-1", "input": "사업 중개, M&A 컨설팅"},
    {"업종": "사업/무형 재산권 중개업", "group": "C-1", "input": "지식재산권 중개, 라이선스 거래"},
]

# C-2: 의료 및 케어 (18개 업종) - 별도 셀에서 처리

# C-3: 기술 및 시공 (13개 업종) - 별도 셀에서 처리

# B+C등급 전체 테스트 케이스 (C-1까지만)
BC_GRADE_TEST_CASES = B1_LEARNING + B2_LEISURE + B3_SPORTS + C1_PROFESSIONAL

print(f"B+C등급(C-1까지) 테스트 케이스 수: {len(BC_GRADE_TEST_CASES)}개 (업종별 2개씩)")

B+C등급(C-1까지) 테스트 케이스 수: 144개 (업종별 2개씩)


In [7]:
# ============================================
# C등급 (계속): C-2, C-3
# D등급: 목적형 소매 및 장비 그룹
# E등급: 인프라 및 특수 목적 그룹
# 각 업종별 2개의 광고 생성용 예시 문장
# ============================================

# C-2: 의료 및 케어 (18개 업종)
C2_MEDICAL = [
    # 종합병원
    {"업종": "종합병원", "group": "C-2", "input": "종합병원, 전문 의료진 진료"},
    {"업종": "종합병원", "group": "C-2", "input": "대학병원, 첨단 의료 시설"},
    # 일반병원
    {"업종": "일반병원", "group": "C-2", "input": "일반병원, 가까운 곳 건강 지킴이"},
    {"업종": "일반병원", "group": "C-2", "input": "병원, 종합 건강검진 안내"},
    # 치과병원
    {"업종": "치과병원", "group": "C-2", "input": "치과병원, 임플란트 전문"},
    {"업종": "치과병원", "group": "C-2", "input": "치과 전문, 치아 교정 상담"},
    # 한방병원
    {"업종": "한방병원", "group": "C-2", "input": "한방병원, 한의학 전문 진료"},
    {"업종": "한방병원", "group": "C-2", "input": "한방 전문, 침 뜸 치료"},
    # 요양병원
    {"업종": "요양병원", "group": "C-2", "input": "요양병원, 어르신 전문 케어"},
    {"업종": "요양병원", "group": "C-2", "input": "요양원, 장기 요양 서비스"},
    # 내과/소아과 의원
    {"업종": "내과/소아과 의원", "group": "C-2", "input": "내과 의원, 건강검진 패키지"},
    {"업종": "내과/소아과 의원", "group": "C-2", "input": "소아과, 영유아 예방접종"},
    # 외과 의원
    {"업종": "외과 의원", "group": "C-2", "input": "외과 의원, 수술 전문"},
    {"업종": "외과 의원", "group": "C-2", "input": "정형외과, 척추 비수술 치료"},
    # 신경/정신과 의원
    {"업종": "신경/정신과 의원", "group": "C-2", "input": "정신건강의학과, 스트레스 상담"},
    {"업종": "신경/정신과 의원", "group": "C-2", "input": "신경과 의원, 두통 진료"},
    # 안과 의원
    {"업종": "안과 의원", "group": "C-2", "input": "안과, 라식 시력교정술"},
    {"업종": "안과 의원", "group": "C-2", "input": "안과 의원, 백내장 수술"},
    # 이비인후과 의원
    {"업종": "이비인후과 의원", "group": "C-2", "input": "이비인후과, 수면무호흡 검사"},
    {"업종": "이비인후과 의원", "group": "C-2", "input": "이비인후과 의원, 비염 치료"},
    # 산부인과 의원
    {"업종": "산부인과 의원", "group": "C-2", "input": "산부인과, 임신 초기 검사"},
    {"업종": "산부인과 의원", "group": "C-2", "input": "산부인과 의원, 여성 건강 상담"},
    # 기타 의원
    {"업종": "기타 의원", "group": "C-2", "input": "의원, 전문 진료 서비스"},
    {"업종": "기타 의원", "group": "C-2", "input": "클리닉, 맞춤 의료 상담"},
    # 치과의원
    {"업종": "치과의원", "group": "C-2", "input": "치과의원, 충치 치료 전문"},
    {"업종": "치과의원", "group": "C-2", "input": "치과, 스케일링 검진"},
    # 한의원
    {"업종": "한의원", "group": "C-2", "input": "한의원, 비만 한방 다이어트"},
    {"업종": "한의원", "group": "C-2", "input": "한의원, 체질 맞춤 처방"},
    # 방사선 진단/병리 검사 의원
    {"업종": "방사선 진단/병리 검사 의원", "group": "C-2", "input": "영상의학과, MRI CT 촬영"},
    {"업종": "방사선 진단/병리 검사 의원", "group": "C-2", "input": "진단검사의학과, 혈액 검사"},
    # 유사 의료업
    {"업종": "유사 의료업", "group": "C-2", "input": "카이로프랙틱, 척추 교정"},
    {"업종": "유사 의료업", "group": "C-2", "input": "추나요법, 체형 교정"},
    # 약국
    {"업종": "약국", "group": "C-2", "input": "약국, 건강기능식품 상담"},
    {"업종": "약국", "group": "C-2", "input": "24시 약국, 야간 조제"},
    # 의료기기 소매업
    {"업종": "의료기기 소매업", "group": "C-2", "input": "의료기기 판매, 혈압계 체온계"},
    {"업종": "의료기기 소매업", "group": "C-2", "input": "의료용품점, 환자용 침대"},
]

# C-3: 기술 및 시공 (13개 업종)
C3_TECHNICAL = [
    # 자동차 정비소
    {"업종": "자동차 정비소", "group": "C-3", "input": "자동차 정비소, 엔진오일 교환"},
    {"업종": "자동차 정비소", "group": "C-3", "input": "카센터, 차량 점검 서비스"},
    # 자동차 세차장
    {"업종": "자동차 세차장", "group": "C-3", "input": "세차장, 프리미엄 손세차"},
    {"업종": "자동차 세차장", "group": "C-3", "input": "셀프 세차장, 자동 세차기"},
    # 모터사이클 수리업
    {"업종": "모터사이클 수리업", "group": "C-3", "input": "오토바이 정비, 전문 수리"},
    {"업종": "모터사이클 수리업", "group": "C-3", "input": "바이크 샵, 오토바이 점검"},
    # 건축 설계 및 관련 서비스업
    {"업종": "건축 설계 및 관련 서비스업", "group": "C-3", "input": "건축 설계, 주택 설계 컨설팅"},
    {"업종": "건축 설계 및 관련 서비스업", "group": "C-3", "input": "건축사무소, 상업시설 설계"},
    # 도시 계획 및 조경 설계 서비스업
    {"업종": "도시 계획 및 조경 설계 서비스업", "group": "C-3", "input": "조경 설계, 정원 디자인"},
    {"업종": "도시 계획 및 조경 설계 서비스업", "group": "C-3", "input": "도시 계획, 개발 컨설팅"},
    # 건물 및 토목 엔지니어링 서비스업
    {"업종": "건물 및 토목 엔지니어링 서비스업", "group": "C-3", "input": "토목 엔지니어링, 구조 설계"},
    {"업종": "건물 및 토목 엔지니어링 서비스업", "group": "C-3", "input": "건축 엔지니어링, 안전 진단"},
    # 환경 관련 엔지니어링 서비스업
    {"업종": "환경 관련 엔지니어링 서비스업", "group": "C-3", "input": "환경 컨설팅, 환경영향평가"},
    {"업종": "환경 관련 엔지니어링 서비스업", "group": "C-3", "input": "환경 엔지니어링, 폐수 처리"},
    # 기타 엔지니어링 서비스업
    {"업종": "기타 엔지니어링 서비스업", "group": "C-3", "input": "엔지니어링 서비스, 기술 컨설팅"},
    {"업종": "기타 엔지니어링 서비스업", "group": "C-3", "input": "설계 서비스, 시스템 설계"},
    # 사업시설 유지·관리 서비스업
    {"업종": "사업시설 유지·관리 서비스업", "group": "C-3", "input": "시설 관리, 건물 유지보수"},
    {"업종": "사업시설 유지·관리 서비스업", "group": "C-3", "input": "관리 대행, 아파트 관리"},
    # 건축물 일반 청소업
    {"업종": "건축물 일반 청소업", "group": "C-3", "input": "건물 청소, 입주 청소 전문"},
    {"업종": "건축물 일반 청소업", "group": "C-3", "input": "사무실 청소, 정기 청소 서비스"},
    # 산업설비, 운송장비 및 공공장소 청소업
    {"업종": "산업설비, 운송장비 및 공공장소 청소업", "group": "C-3", "input": "산업 청소, 공장 청소 서비스"},
    {"업종": "산업설비, 운송장비 및 공공장소 청소업", "group": "C-3", "input": "특수 청소, 대형 시설 청소"},
    # 소독, 구충 및 방제 서비스업
    {"업종": "소독, 구충 및 방제 서비스업", "group": "C-3", "input": "방역 소독, 바퀴벌레 퇴치"},
    {"업종": "소독, 구충 및 방제 서비스업", "group": "C-3", "input": "해충 방제, 쥐 퇴치 서비스"},
    # 조경 유지·관리 서비스업
    {"업종": "조경 유지·관리 서비스업", "group": "C-3", "input": "조경 관리, 정원 가꾸기"},
    {"업종": "조경 유지·관리 서비스업", "group": "C-3", "input": "조경 유지, 잔디 관리 서비스"},
]

# D-1: 중장비 및 부품 (13개 업종)
D1_HEAVY_EQUIPMENT = [
    # 타이어 소매업
    {"업종": "타이어 소매업", "group": "D-1", "input": "타이어 전문점, 사계절 타이어"},
    {"업종": "타이어 소매업", "group": "D-1", "input": "타이어 매장, 교체 서비스"},
    # 자동차 부품 소매업
    {"업종": "자동차 부품 소매업", "group": "D-1", "input": "자동차 부품, 순정 브레이크패드"},
    {"업종": "자동차 부품 소매업", "group": "D-1", "input": "부품 전문점, 차량 소모품"},
    # 모터사이클 및 부품 소매업
    {"업종": "모터사이클 및 부품 소매업", "group": "D-1", "input": "오토바이 매장, 바이크 액세서리"},
    {"업종": "모터사이클 및 부품 소매업", "group": "D-1", "input": "모터사이클 부품, 헬멧 판매"},
    # 철물/공구 소매업
    {"업종": "철물/공구 소매업", "group": "D-1", "input": "철물점, 공구 세트 특가"},
    {"업종": "철물/공구 소매업", "group": "D-1", "input": "공구 전문점, 전동 공구"},
    # 벽지/장판/마루 소매업
    {"업종": "벽지/장판/마루 소매업", "group": "D-1", "input": "바닥재 전문, 마루 시공"},
    {"업종": "벽지/장판/마루 소매업", "group": "D-1", "input": "벽지 매장, 도배 자재"},
    # 건설/건축자재 소매업
    {"업종": "건설/건축자재 소매업", "group": "D-1", "input": "건축자재 판매, 시멘트 벽돌"},
    {"업종": "건설/건축자재 소매업", "group": "D-1", "input": "건자재 도매, 목재 판매"},
    # 기타 건설/건축자재 소매업
    {"업종": "기타 건설/건축자재 소매업", "group": "D-1", "input": "건설자재, 철근 판매"},
    {"업종": "기타 건설/건축자재 소매업", "group": "D-1", "input": "건축 소재, 단열재 판매"},
    # 전기용품/조명장치 소매업
    {"업종": "전기용품/조명장치 소매업", "group": "D-1", "input": "전기용품 가게, LED 조명"},
    {"업종": "전기용품/조명장치 소매업", "group": "D-1", "input": "조명 전문점, 인테리어 조명"},
    # 건설기계/장비 대여업
    {"업종": "건설기계/장비 대여업", "group": "D-1", "input": "중장비 대여, 굴삭기 렌탈"},
    {"업종": "건설기계/장비 대여업", "group": "D-1", "input": "건설장비 임대, 포클레인 대여"},
    # 기타 산업용 기계/장비 대여업
    {"업종": "기타 산업용 기계/장비 대여업", "group": "D-1", "input": "산업기계 대여, 지게차 렌탈"},
    {"업종": "기타 산업용 기계/장비 대여업", "group": "D-1", "input": "장비 임대, 발전기 대여"},
    # 기타 운송장비 대여업
    {"업종": "기타 운송장비 대여업", "group": "D-1", "input": "화물차 대여, 트럭 렌탈"},
    {"업종": "기타 운송장비 대여업", "group": "D-1", "input": "운송장비 임대, 밴 대여"},
    # 가정용 연료 소매업
    {"업종": "가정용 연료 소매업", "group": "D-1", "input": "연료 판매, 등유 배달"},
    {"업종": "가정용 연료 소매업", "group": "D-1", "input": "가정용 연료, 보일러 기름"},
    # 포장/충전업
    {"업종": "포장/충전업", "group": "D-1", "input": "포장 서비스, 산업용 가스 충전"},
    {"업종": "포장/충전업", "group": "D-1", "input": "충전 서비스, LPG 충전"},
]

# D-2: 일반 기기 (14개 업종)
D2_GENERAL_EQUIPMENT = [
    # 컴퓨터/소프트웨어 소매업
    {"업종": "컴퓨터/소프트웨어 소매업", "group": "D-2", "input": "컴퓨터 매장, 노트북 판매"},
    {"업종": "컴퓨터/소프트웨어 소매업", "group": "D-2", "input": "소프트웨어 전문, 정품 SW"},
    # 핸드폰 소매업
    {"업종": "핸드폰 소매업", "group": "D-2", "input": "핸드폰 매장, 스마트폰 개통"},
    {"업종": "핸드폰 소매업", "group": "D-2", "input": "휴대폰 판매, 최신 기종"},
    # 가전제품 소매업
    {"업종": "가전제품 소매업", "group": "D-2", "input": "가전제품 매장, 에어컨 특가"},
    {"업종": "가전제품 소매업", "group": "D-2", "input": "전자제품 할인점, 냉장고 판매"},
    # 사무기기 소매업
    {"업종": "사무기기 소매업", "group": "D-2", "input": "사무기기 판매, 프린터 복합기"},
    {"업종": "사무기기 소매업", "group": "D-2", "input": "오피스 장비, 사무용품"},
    # 컴퓨터/노트북/프린터 수리업
    {"업종": "컴퓨터/노트북/프린터 수리업", "group": "D-2", "input": "컴퓨터 수리, SSD 업그레이드"},
    {"업종": "컴퓨터/노트북/프린터 수리업", "group": "D-2", "input": "노트북 AS, 수리 전문점"},
    # 컴퓨터/사무기기 대여업
    {"업종": "컴퓨터/사무기기 대여업", "group": "D-2", "input": "복합기 렌탈, 사무실 임대"},
    {"업종": "컴퓨터/사무기기 대여업", "group": "D-2", "input": "프린터 대여, 단기 렌탈"},
    # 핸드폰/통신장비 수리업
    {"업종": "핸드폰/통신장비 수리업", "group": "D-2", "input": "폰 수리 전문, 액정 교체"},
    {"업종": "핸드폰/통신장비 수리업", "group": "D-2", "input": "스마트폰 수리, 당일 AS"},
    # 가전제품 수리업
    {"업종": "가전제품 수리업", "group": "D-2", "input": "가전 수리, 출장 서비스"},
    {"업종": "가전제품 수리업", "group": "D-2", "input": "TV 수리, 냉장고 AS"},
    # 자동차 대여업
    {"업종": "자동차 대여업", "group": "D-2", "input": "렌터카, 장기 렌트 프로모션"},
    {"업종": "자동차 대여업", "group": "D-2", "input": "자동차 렌탈, 단기 대여"},
    # 그 외 기타 개인/가정용품 수리업
    {"업종": "그 외 기타 개인/가정용품 수리업", "group": "D-2", "input": "수리 서비스, 가정용품 AS"},
    {"업종": "그 외 기타 개인/가정용품 수리업", "group": "D-2", "input": "생활용품 수리, 출장 서비스"},
    # 그 외 기타 개인/가정용품 대여업
    {"업종": "그 외 기타 개인/가정용품 대여업", "group": "D-2", "input": "용품 대여, 가정용품 렌탈"},
    {"업종": "그 외 기타 개인/가정용품 대여업", "group": "D-2", "input": "생활용품 임대, 단기 대여"},
    # 세탁소
    {"업종": "세탁소", "group": "D-2", "input": "세탁소, 드라이클리닝 할인"},
    {"업종": "세탁소", "group": "D-2", "input": "세탁 전문, 이불 세탁"},
    # 셀프 빨래방
    {"업종": "셀프 빨래방", "group": "D-2", "input": "셀프 빨래방, 24시간 코인 세탁"},
    {"업종": "셀프 빨래방", "group": "D-2", "input": "코인 세탁소, 대형 빨래 가능"},
    # 중고 상품 소매업
    {"업종": "중고 상품 소매업", "group": "D-2", "input": "중고 매장, 가전 매입 판매"},
    {"업종": "중고 상품 소매업", "group": "D-2", "input": "중고품 전문, 명품 중고"},
]

# D-3: 취미 및 라이프스타일 소매 (18개 업종)
D3_HOBBY_LIFESTYLE = [
    # 악기 소매업
    {"업종": "악기 소매업", "group": "D-3", "input": "악기점, 기타와 피아노 판매"},
    {"업종": "악기 소매업", "group": "D-3", "input": "악기 전문점, 드럼 세트"},
    # 음반/비디오물 소매업
    {"업종": "음반/비디오물 소매업", "group": "D-3", "input": "레코드샵, LP 앨범 판매"},
    {"업종": "음반/비디오물 소매업", "group": "D-3", "input": "음반 매장, DVD 판매"},
    # 음반/비디오물 대여업
    {"업종": "음반/비디오물 대여업", "group": "D-3", "input": "비디오 대여, DVD 렌탈"},
    {"업종": "음반/비디오물 대여업", "group": "D-3", "input": "영화 대여, 신작 DVD"},
    # 담배/전자담배 소매업
    {"업종": "담배/전자담배 소매업", "group": "D-3", "input": "전자담배 전문점, 액상 판매"},
    {"업종": "담배/전자담배 소매업", "group": "D-3", "input": "담배 가게, 다양한 담배"},
    # 실/섬유제품 소매업
    {"업종": "실/섬유제품 소매업", "group": "D-3", "input": "원단 매장, 섬유 판매"},
    {"업종": "실/섬유제품 소매업", "group": "D-3", "input": "실 전문점, 뜨개질 재료"},
    # 주류 소매업
    {"업종": "주류 소매업", "group": "D-3", "input": "와인샵, 프리미엄 와인"},
    {"업종": "주류 소매업", "group": "D-3", "input": "주류 판매, 위스키 전문"},
    # 예술품 소매업
    {"업종": "예술품 소매업", "group": "D-3", "input": "갤러리, 미술 작품 판매"},
    {"업종": "예술품 소매업", "group": "D-3", "input": "예술품 매장, 조각 작품"},
    # 기념품점
    {"업종": "기념품점", "group": "D-3", "input": "기념품샵, 여행 기념품"},
    {"업종": "기념품점", "group": "D-3", "input": "선물 가게, 특산품 판매"},
    # 서점
    {"업종": "서점", "group": "D-3", "input": "서점, 베스트셀러 도서"},
    {"업종": "서점", "group": "D-3", "input": "독립 서점, 아늑한 독서 공간"},
    # 문구/회화용품 소매업
    {"업종": "문구/회화용품 소매업", "group": "D-3", "input": "문구점, 다이어리와 펜"},
    {"업종": "문구/회화용품 소매업", "group": "D-3", "input": "화방, 미술 용품 전문"},
    # 장난감 소매업
    {"업종": "장난감 소매업", "group": "D-3", "input": "장난감 가게, 레고 신제품"},
    {"업종": "장난감 소매업", "group": "D-3", "input": "완구점, 아이들 선물"},
    # 운동용품 소매업
    {"업종": "운동용품 소매업", "group": "D-3", "input": "운동용품 매장, 스포츠 장비"},
    {"업종": "운동용품 소매업", "group": "D-3", "input": "스포츠 전문점, 운동복 판매"},
    # 자전거 소매업
    {"업종": "자전거 소매업", "group": "D-3", "input": "자전거 매장, 로드바이크"},
    {"업종": "자전거 소매업", "group": "D-3", "input": "자전거 전문점, MTB 판매"},
    # 시계/귀금속/악기 수리업
    {"업종": "시계/귀금속/악기 수리업", "group": "D-3", "input": "시계 수리, 명품 시계 AS"},
    {"업종": "시계/귀금속/악기 수리업", "group": "D-3", "input": "악기 수리, 기타 조율"},
    # 가축 사료 소매업
    {"업종": "가축 사료 소매업", "group": "D-3", "input": "사료 판매, 동물 먹이"},
    {"업종": "가축 사료 소매업", "group": "D-3", "input": "축산 사료, 도매 판매"},
    # 곡물/곡분 소매업
    {"업종": "곡물/곡분 소매업", "group": "D-3", "input": "쌀 전문점, 햅쌀 판매"},
    {"업종": "곡물/곡분 소매업", "group": "D-3", "input": "곡물 판매, 잡곡 세트"},
    # 그 외 기타 종합 소매업
    {"업종": "그 외 기타 종합 소매업", "group": "D-3", "input": "종합 쇼핑몰, 다양한 상품"},
    {"업종": "그 외 기타 종합 소매업", "group": "D-3", "input": "백화점, 프리미엄 쇼핑"},
    # 그 외 기타 상품 전문 소매업
    {"업종": "그 외 기타 상품 전문 소매업", "group": "D-3", "input": "전문 소매점, 특수 상품"},
    {"업종": "그 외 기타 상품 전문 소매업", "group": "D-3", "input": "특화 매장, 니치 상품"},
]

# E-1: 편의 및 인프라 (3개 업종)
E1_INFRASTRUCTURE = [
    # 주유소
    {"업종": "주유소", "group": "E-1", "input": "주유소, 리터당 할인 이벤트"},
    {"업종": "주유소", "group": "E-1", "input": "셀프 주유소, 24시간 운영"},
    # 가스 충전소
    {"업종": "가스 충전소", "group": "E-1", "input": "LPG 충전소, 차량 가스 충전"},
    {"업종": "가스 충전소", "group": "E-1", "input": "가스 충전, 안전한 충전 서비스"},
    # 얼음 소매업
    {"업종": "얼음 소매업", "group": "E-1", "input": "얼음 배달, 대용량 얼음 판매"},
    {"업종": "얼음 소매업", "group": "E-1", "input": "얼음 도매, 여름 얼음 대량"},
]

# E-2: 예절 및 의례 (2개 업종)
E2_CEREMONY = [
    # 장례식장
    {"업종": "장례식장", "group": "E-2", "input": "장례식장, 정성을 다하는 추모"},
    {"업종": "장례식장", "group": "E-2", "input": "장례 서비스, 엄숙한 장례"},
    # 화장터/묘지/납골당
    {"업종": "화장터/묘지/납골당", "group": "E-2", "input": "납골당 분양, 평화로운 안식처"},
    {"업종": "화장터/묘지/납골당", "group": "E-2", "input": "수목장, 자연 속 추모"},
]

# C-2, C-3, D, E등급 전체 테스트 케이스
CDE_GRADE_TEST_CASES = C2_MEDICAL + C3_TECHNICAL + D1_HEAVY_EQUIPMENT + D2_GENERAL_EQUIPMENT + D3_HOBBY_LIFESTYLE + E1_INFRASTRUCTURE + E2_CEREMONY

print(f"C-2,C-3,D,E등급 테스트 케이스 수: {len(CDE_GRADE_TEST_CASES)}개 (업종별 2개씩)")

C-2,C-3,D,E등급 테스트 케이스 수: 162개 (업종별 2개씩)


In [8]:
# 전체 테스트 케이스 통합
ALL_TEST_CASES = S_GRADE_TEST_CASES + A_GRADE_TEST_CASES + BC_GRADE_TEST_CASES + CDE_GRADE_TEST_CASES

print(f"\n=== 테스트 케이스 요약 ===")
print(f"S등급: {len(S_GRADE_TEST_CASES)}개 (6개 그룹)")
print(f"A등급: {len(A_GRADE_TEST_CASES)}개 (4개 그룹)")
print(f"B+C등급: {len(BC_GRADE_TEST_CASES)}개 (6개 그룹)")
print(f"D+E등급: {len(CDE_GRADE_TEST_CASES)}개 (5개 그룹)")
print(f"총 테스트 케이스: {len(ALL_TEST_CASES)}개 (21개 그룹)")


=== 테스트 케이스 요약 ===
S등급: 106개 (6개 그룹)
A등급: 82개 (4개 그룹)
B+C등급: 144개 (6개 그룹)
D+E등급: 162개 (5개 그룹)
총 테스트 케이스: 494개 (21개 그룹)


## 3. 업종 인식 평가 함수 정의

In [9]:
# ============================================
# 그룹 코드 매핑 테이블 (YAML 파일 키 값 기준 수정)
# ============================================

# 테스트 데이터 그룹명 -> YAML 하위 그룹 코드 매핑
GROUP_TO_CODE = {
    # S등급
    "S-1": "s1_hot_cooking",
    "S-2": "s2_freshness",
    "S-3": "s3_emotional",
    "S-4": "s4_korean_homestyle",
    "S-5": "s5_global_variety",
    "S-6": "s6_nightlife",
    # A등급
    "A-1": "a1_beauty",
    "A-2": "a2_wellness",
    "A-3": "a3_fashion",
    "A-4": "a4_delicate_care",
    # B등급
    "B-1": "b1_learning",
    "B-2": "b2_leisure",
    "B-3": "b3_sports",
    # C등급
    "C-1": "c1_office", 
    "C-2": "c2_medical",
    "C-3": "c3_technical",
    # D등급
    "D-1": "d1_equipment",
    "D-2": "d2_appliances",
    "D-3": "d3_hobby_lifestyle",
    # E등급
    "E-1": "e1_infrastructure",
    "E-2": "e2_ceremony",
}

# 역매핑 (코드 -> 그룹명)
CODE_TO_GROUP = {v: k for k, v in GROUP_TO_CODE.items()}

print(f"매핑 테이블 로드 완료: {len(GROUP_TO_CODE)}개 그룹")

매핑 테이블 로드 완료: 21개 그룹


In [10]:
# ============================================
# 업종 인식 평가 함수
# ============================================

def evaluate_industry_detection(test_cases: List[Dict], prompt_manager: PromptTemplateManager) -> List[Dict]:
    """
    테스트 케이스에 대해 업종 인식을 수행하고 결과를 반환
    
    Args:
        test_cases: 테스트 케이스 리스트 [{"group": "S-1", "input": "..."}]
        prompt_manager: PromptTemplateManager 인스턴스
        
    Returns:
        평가 결과 리스트
    """
    results = []
    
    for case in test_cases:
        expected_group = case["group"]
        user_input = case["input"]
        expected_code = GROUP_TO_CODE.get(expected_group, "unknown")
        
        # 업종 인식 수행
        try:
            detected_code = prompt_manager._detect_industry(user_input)
            detected_group = CODE_TO_GROUP.get(detected_code, detected_code)
        except Exception as e:
            detected_code = "error"
            detected_group = f"Error: {str(e)}"
        
        # 일치 여부 판단
        is_match = (expected_code == detected_code)
        
        results.append({
            "input": user_input,
            "expected_group": expected_group,
            "expected_code": expected_code,
            "detected_code": detected_code,
            "detected_group": detected_group,
            "is_match": is_match
        })
    
    return results


def calculate_accuracy_by_group(results: List[Dict]) -> Dict:
    """
    그룹별 정확도 계산
    
    Args:
        results: 평가 결과 리스트
        
    Returns:
        그룹별 정확도 딕셔너리
    """
    group_stats = {}
    
    for result in results:
        group = result["expected_group"]
        if group not in group_stats:
            group_stats[group] = {"total": 0, "correct": 0}
        
        group_stats[group]["total"] += 1
        if result["is_match"]:
            group_stats[group]["correct"] += 1
    
    # 정확도 계산
    accuracy_by_group = {}
    for group, stats in group_stats.items():
        accuracy = (stats["correct"] / stats["total"]) * 100 if stats["total"] > 0 else 0
        accuracy_by_group[group] = {
            "total": stats["total"],
            "correct": stats["correct"],
            "accuracy": round(accuracy, 1)
        }
    
    return accuracy_by_group


def get_failed_cases(results: List[Dict]) -> List[Dict]:
    """
    인식 실패 케이스 추출
    
    Args:
        results: 평가 결과 리스트
        
    Returns:
        실패 케이스 리스트
    """
    return [r for r in results if not r["is_match"]]


print("평가 함수 정의 완료!")

평가 함수 정의 완료!


## 4. 업종 인식 테스트 실행

In [11]:
# 전체 테스트 케이스에 대해 업종 인식 평가 실행
print("업종 인식 평가 시작...")
print(f"총 테스트 케이스: {len(ALL_TEST_CASES)}개\n")

# 평가 실행
all_results = evaluate_industry_detection(ALL_TEST_CASES, prompt_manager)

# 전체 정확도 계산
total_correct = sum(1 for r in all_results if r["is_match"])
total_accuracy = (total_correct / len(all_results)) * 100

print(f"평가 완료!")
print(f"전체 정확도: {total_correct}/{len(all_results)} ({total_accuracy:.1f}%)")

업종 인식 평가 시작...
총 테스트 케이스: 494개

평가 완료!
전체 정확도: 291/494 (58.9%)


## 5. 업종 인식 일치율 평가 결과 정리

### 5.1 그룹별 정확도 테이블

In [12]:
# 그룹별 정확도 계산
accuracy_by_group = calculate_accuracy_by_group(all_results)

# DataFrame으로 변환
accuracy_df = pd.DataFrame([
    {
        "그룹": group,
        "정확 개수": stats["correct"],
        "총 개수": stats["total"],
        "정확도(%)": stats["accuracy"]
    }
    for group, stats in sorted(accuracy_by_group.items())
])

# 등급별 정렬을 위한 순서 정의
group_order = ["S-1", "S-2", "S-3", "S-4", "S-5", "S-6",
               "A-1", "A-2", "A-3", "A-4",
               "B-1", "B-2", "B-3",
               "C-1", "C-2", "C-3",
               "D-1", "D-2", "D-3",
               "E-1", "E-2"]

accuracy_df["그룹"] = pd.Categorical(accuracy_df["그룹"], categories=group_order, ordered=True)
accuracy_df = accuracy_df.sort_values("그룹").reset_index(drop=True)

print("=" * 50)
print("그룹별 업종 인식 정확도")
print("=" * 50)
print(accuracy_df.to_string(index=False))
print("=" * 50)

그룹별 업종 인식 정확도
 그룹  정확 개수  총 개수  정확도(%)
S-1     25    34    73.5
S-2     14    16    87.5
S-3     21    22    95.5
S-4     17    18    94.4
S-5      1     8    12.5
S-6      1     8    12.5
A-1     11    12    91.7
A-2     10    12    83.3
A-3     29    36    80.6
A-4      9    22    40.9
B-1     23    36    63.9
B-2     11    18    61.1
B-3     21    38    55.3
C-1     24    52    46.2
C-2     23    36    63.9
C-3     11    26    42.3
D-1      7    26    26.9
D-2     16    28    57.1
D-3     10    36    27.8
E-1      4     6    66.7
E-2      3     4    75.0


In [13]:
# 등급별 정확도 요약
grade_summary = {
    "S등급": ["S-1", "S-2", "S-3", "S-4", "S-5", "S-6"],
    "A등급": ["A-1", "A-2", "A-3", "A-4"],
    "B등급": ["B-1", "B-2", "B-3"],
    "C등급": ["C-1", "C-2", "C-3"],
    "D등급": ["D-1", "D-2", "D-3"],
    "E등급": ["E-1", "E-2"]
}

print("\n" + "=" * 50)
print("등급별 정확도 요약")
print("=" * 50)

grade_results = []
for grade, groups in grade_summary.items():
    total = sum(accuracy_by_group[g]["total"] for g in groups if g in accuracy_by_group)
    correct = sum(accuracy_by_group[g]["correct"] for g in groups if g in accuracy_by_group)
    accuracy = (correct / total) * 100 if total > 0 else 0
    grade_results.append({
        "등급": grade,
        "정확 개수": correct,
        "총 개수": total,
        "정확도(%)": round(accuracy, 1)
    })
    print(f"{grade}: {correct}/{total} ({accuracy:.1f}%)")

print("=" * 50)
print(f"\n전체 정확도: {total_correct}/{len(all_results)} ({total_accuracy:.1f}%)")


등급별 정확도 요약
S등급: 79/106 (74.5%)
A등급: 59/82 (72.0%)
B등급: 55/92 (59.8%)
C등급: 58/114 (50.9%)
D등급: 33/90 (36.7%)
E등급: 7/10 (70.0%)

전체 정확도: 291/494 (58.9%)


### 5.2 인식 실패 케이스 상세 분석

In [14]:
# 실패 케이스 추출
failed_cases = get_failed_cases(all_results)

print(f"\n=== 인식 실패 케이스: {len(failed_cases)}개 ===")
print("\n")

# 실패 케이스를 DataFrame으로 변환
if failed_cases:
    failed_df = pd.DataFrame([
        {
            "입력": case["input"][:40] + "..." if len(case["input"]) > 40 else case["input"],
            "예상 그룹": case["expected_group"],
            "인식 그룹": case["detected_group"],
            "인식 코드": case["detected_code"]
        }
        for case in failed_cases
    ])
    
    # 예상 그룹별로 정렬
    failed_df["예상 그룹"] = pd.Categorical(failed_df["예상 그룹"], categories=group_order, ordered=True)
    failed_df = failed_df.sort_values("예상 그룹").reset_index(drop=True)
    
    print(failed_df.to_string(index=False))
else:
    print("모든 케이스가 정확하게 인식되었습니다!")


=== 인식 실패 케이스: 203개 ===


                   입력 예상 그룹 인식 그룹               인식 코드
껍데기 맛집 홍보, 바삭한 돼지 껍데기   S-1   S-3        s3_emotional
    갈비찜 전문점, 부드러운 갈비살   S-1   S-4 s4_korean_homestyle
    곱창 전골, 얼큰한 국물이 일품   S-1   S-4 s4_korean_homestyle
    찜닭 맛집, 매콤달콤 양념 찜닭   S-1   S-3        s3_emotional
  마라탕 전문점, 얼얼하게 매운 국물   S-1   S-4 s4_korean_homestyle
 패밀리 레스토랑, 온 가족 외식 코스   S-1   S-3        s3_emotional
  일본식 카레 전문, 진한 카레 소스   S-1   S-4 s4_korean_homestyle
     사내 식당, 건강한 한끼 제공   S-1   S-3        s3_emotional
   쌀국수 전문점, 허브와 숙주 가득   S-1   S-4 s4_korean_homestyle
   복국 맛집, 담백한 복국 한 그릇   S-2   S-4 s4_korean_homestyle
    정육 직판장, 신선한 고기 배달   S-2   S-1      s1_hot_cooking
     슈퍼마켓, 신선식품 할인 행사   S-3   S-2        s2_freshness
        식료품점, 건강한 먹거리   S-4   S-3        s3_emotional
    일식당, 오코노미야키와 타코야끼   S-5   S-3        s3_emotional
      이자카야, 일본 가정식 요리   S-5   S-4 s4_korean_homestyle
      양식당, 리조또와 뇨끼 세트   S-5   S-3        s3_emotional
     유러피안 레스토랑, 코스 요리   S-5   S-3        s3_emotional
 

In [15]:
# 실패 패턴 분석: 어떤 그룹으로 잘못 인식되는 경향이 있는지
if failed_cases:
    print("\n=== 실패 패턴 분석 ===")
    print("(예상 그룹 -> 잘못 인식된 그룹 빈도)\n")
    
    confusion_matrix = {}
    for case in failed_cases:
        expected = case["expected_group"]
        detected = case["detected_group"]
        key = f"{expected} -> {detected}"
        confusion_matrix[key] = confusion_matrix.get(key, 0) + 1
    
    # 빈도순 정렬
    sorted_confusion = sorted(confusion_matrix.items(), key=lambda x: x[1], reverse=True)
    
    for pattern, count in sorted_confusion:
        print(f"  {pattern}: {count}건")


=== 실패 패턴 분석 ===
(예상 그룹 -> 잘못 인식된 그룹 빈도)

  C-1 -> S-3: 16건
  B-3 -> S-2: 8건
  D-3 -> S-4: 8건
  C-1 -> S-4: 7건
  C-3 -> S-3: 7건
  D-1 -> S-4: 7건
  A-4 -> A-3: 6건
  B-1 -> S-3: 6건
  C-2 -> S-3: 6건
  D-3 -> S-2: 6건
  S-1 -> S-4: 5건
  B-3 -> S-3: 5건
  C-2 -> S-4: 5건
  S-1 -> S-3: 4건
  S-5 -> S-4: 4건
  S-6 -> S-3: 4건
  A-3 -> S-4: 4건
  B-1 -> D-3: 4건
  C-1 -> S-2: 4건
  C-3 -> C-1: 4건
  D-1 -> A-3: 4건
  D-2 -> S-4: 4건
  D-3 -> S-3: 4건
  S-5 -> S-3: 3건
  A-4 -> S-4: 3건
  B-2 -> B-1: 3건
  C-3 -> S-4: 3건
  D-1 -> C-3: 3건
  D-2 -> A-3: 3건
  D-3 -> A-3: 3건
  A-2 -> S-2: 2건
  A-3 -> S-3: 2건
  A-4 -> S-3: 2건
  B-1 -> S-4: 2건
  B-2 -> S-2: 2건
  B-3 -> A-3: 2건
  D-1 -> S-3: 2건
  D-1 -> S-2: 2건
  D-2 -> A-4: 2건
  D-3 -> A-4: 2건
  D-3 -> D-1: 2건
  E-1 -> S-3: 2건
  S-2 -> S-4: 1건
  S-2 -> S-1: 1건
  S-3 -> S-2: 1건
  S-4 -> S-3: 1건
  S-6 -> S-4: 1건
  S-6 -> S-1: 1건
  S-6 -> D-1: 1건
  A-1 -> S-4: 1건
  A-3 -> S-2: 1건
  A-4 -> C-2: 1건
  A-4 -> S-2: 1건
  B-1 -> C-3: 1건
  B-2 -> S-4: 1건
  B-2 -> S-3: 1건
  B-

## 6. API를 이용한 실패 원인 분석 및 개선안 보고서 생성

업종 인식 실패 케이스를 GPT API로 분석하여 `industries.yaml` 개선 방안을 도출합니다.

In [16]:
# ============================================
# GPT API를 이용한 실패 원인 분석 함수 (JSON 형식)
# ============================================

def analyze_failure_with_gpt(failed_cases: List[Dict], accuracy_by_group: Dict) -> Dict:
    """
    GPT API를 사용하여 업종 인식 실패 원인 분석 및 개선안 도출 (JSON 형식)
    
    Args:
        failed_cases: 실패 케이스 리스트
        accuracy_by_group: 그룹별 정확도 딕셔너리
        
    Returns:
        그룹별 분석 결과 딕셔너리
    """
    if not failed_cases:
        return {}
    
    # 그룹별로 실패 케이스 분류
    failures_by_group = {}
    for case in failed_cases:
        group = case["expected_group"]
        if group not in failures_by_group:
            failures_by_group[group] = []
        failures_by_group[group].append(case)
    
    suggestions_by_group = {}
    
    for group, cases in failures_by_group.items():
        # 해당 그룹의 실패 케이스 요약
        failure_summary = []
        for case in cases[:5]:  # 최대 5개까지만 분석
            summary_item = (
                f'- 입력: "{case["input"]}"\n'
                f'  예상: {case["expected_group"]} ({case["expected_code"]})\n'
                f'  인식: {case["detected_group"]} ({case["detected_code"]})'
            )
            failure_summary.append(summary_item)
        
        failure_text = "".join(failure_summary)
        
        # GPT 프롬프트 구성
        system_prompt = """당신은 소상공인 업종 분류 시스템의 전문가입니다.
업종 인식 실패 케이스를 분석하고, industries.yaml 파일의 korean_keywords 설정을 개선하는 방안을 JSON 형식으로 제안해주세요.

반드시 아래 JSON 형식으로만 응답하세요:
{
    "analysis": {
        "why_failed": "실패 원인 분석",
        "confusion_pattern": "혼동 패턴 설명",
        "common_issues": "공통적인 문제점"
    },
    "improvements": {
        "korean_keywords": {
            "add": ["추가할 키워드1", "추가할 키워드2"],
            "remove": ["제거할 키워드"],
            "reason": "키워드 변경 이유"
        },
        "detection_strategy": {
            "current_issue": "현재 감지 전략의 문제점",
            "suggestion": "개선 제안",
            "reason": "개선이 필요한 이유"
        }
    },
    "priority": "높음/중간/낮음",
    "summary": "전체 요약"
}"""

        user_prompt = f"""다음은 {group} 그룹의 업종 인식 실패 케이스입니다.

## 실패 케이스
{failure_text}

## 그룹 정보
- 그룹: {group}
- 코드: {GROUP_TO_CODE.get(group, 'unknown')}
- 총 테스트: {accuracy_by_group.get(group, {}).get('total', 0)}개
- 정확 인식: {accuracy_by_group.get(group, {}).get('correct', 0)}개
- 정확도: {accuracy_by_group.get(group, {}).get('accuracy', 0)}%

위 실패 케이스들을 분석하고 korean_keywords 개선 방안을 JSON 형식으로 제시해주세요."""

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.7,
                max_tokens=2000,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            suggestions_by_group[group] = {
                "group": group,
                "failed_cases_count": len(cases),
                "accuracy": accuracy_by_group.get(group, {}).get('accuracy', 0),
                **result
            }
            print(f"  {group} 분석 완료 (실패 {len(cases)}건)")
            
        except Exception as e:
            suggestions_by_group[group] = {
                "group": group,
                "failed_cases_count": len(cases),
                "error": str(e)
            }
            print(f"  {group} 분석 실패: {str(e)}")
    
    return suggestions_by_group


print("분석 함수 정의 완료!")

분석 함수 정의 완료!


In [17]:
# ============================================
# JSON 형식 개선안 보고서 생성 함수
# ============================================

def generate_improvement_report_json(
    all_results: List[Dict],
    failed_cases: List[Dict],
    accuracy_by_group: Dict,
    suggestions_by_group: Dict
) -> Dict:
    """
    업종 인식률 개선안 JSON 보고서 생성
    
    Args:
        all_results: 전체 평가 결과
        failed_cases: 실패 케이스
        accuracy_by_group: 그룹별 정확도
        suggestions_by_group: GPT 분석 결과
        
    Returns:
        JSON 형식의 보고서 딕셔너리
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    total_correct = sum(1 for r in all_results if r["is_match"])
    total_accuracy = (total_correct / len(all_results)) * 100 if all_results else 0
    
    # 등급별 정확도 계산
    grade_accuracy = {}
    for grade, groups in grade_summary.items():
        total = sum(accuracy_by_group[g]["total"] for g in groups if g in accuracy_by_group)
        correct = sum(accuracy_by_group[g]["correct"] for g in groups if g in accuracy_by_group)
        accuracy = (correct / total) * 100 if total > 0 else 0
        grade_accuracy[grade] = {
            "correct": correct,
            "total": total,
            "accuracy": round(accuracy, 1)
        }
    
    # 실패 케이스 상세 정보
    failed_cases_detail = [
        {
            "input": case["input"],
            "expected_group": case["expected_group"],
            "expected_code": case["expected_code"],
            "detected_group": case["detected_group"],
            "detected_code": case["detected_code"]
        }
        for case in failed_cases
    ]
    
    # JSON 보고서 구성
    report = {
        "report_type": "업종 인식률 개선사항",
        "target_file": "src/generation/image_generation/prompt/config/industries.yaml",
        "analysis_date": timestamp,
        "summary": {
            "total_test_cases": len(all_results),
            "correct_detections": total_correct,
            "failed_detections": len(failed_cases),
            "overall_accuracy": round(total_accuracy, 1)
        },
        "grade_accuracy": grade_accuracy,
        "group_accuracy": {
            group: {
                "correct": stats["correct"],
                "total": stats["total"],
                "accuracy": stats["accuracy"]
            }
            for group, stats in sorted(accuracy_by_group.items())
        },
        "failed_cases": failed_cases_detail,
        "groups_analyzed": len(suggestions_by_group),
        "suggestions_by_group": suggestions_by_group
    }
    
    return report


print("JSON 보고서 생성 함수 정의 완료!")

JSON 보고서 생성 함수 정의 완료!


In [18]:
# ============================================
# GPT 분석 실행
# ============================================

print("GPT API를 이용한 실패 원인 분석 중...")
print(f"분석 대상: {len(failed_cases)}건의 실패 케이스")

# GPT 분석 실행 (실패 케이스가 있는 그룹만)
if failed_cases:
    suggestions_by_group = analyze_failure_with_gpt(failed_cases, accuracy_by_group)
    print(f"분석 완료! 총 {len(suggestions_by_group)}개 그룹 분석됨")
else:
    suggestions_by_group = {}
    print("실패 케이스가 없어 분석을 건너뜁니다.")

GPT API를 이용한 실패 원인 분석 중...
분석 대상: 203건의 실패 케이스
  S-1 분석 완료 (실패 9건)
  S-2 분석 완료 (실패 2건)
  S-3 분석 완료 (실패 1건)
  S-4 분석 완료 (실패 1건)
  S-5 분석 완료 (실패 7건)
  S-6 분석 완료 (실패 7건)
  A-1 분석 완료 (실패 1건)
  A-2 분석 완료 (실패 2건)
  A-3 분석 완료 (실패 7건)
  A-4 분석 완료 (실패 13건)
  B-1 분석 완료 (실패 13건)
  B-2 분석 완료 (실패 7건)
  B-3 분석 완료 (실패 17건)
  C-1 분석 완료 (실패 28건)
  C-2 분석 완료 (실패 13건)
  C-3 분석 완료 (실패 15건)
  D-1 분석 완료 (실패 19건)
  D-2 분석 완료 (실패 12건)
  D-3 분석 완료 (실패 26건)
  E-1 분석 완료 (실패 2건)
  E-2 분석 완료 (실패 1건)
분석 완료! 총 21개 그룹 분석됨


In [19]:
# JSON 보고서 생성 및 저장
report = generate_improvement_report_json(
    all_results=all_results,
    failed_cases=failed_cases,
    accuracy_by_group=accuracy_by_group,
    suggestions_by_group=suggestions_by_group
)

# 타임스탬프를 포함한 파일명 생성
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
report_filename = f"{timestamp}_업종_인식률_개선안.json"
report_path = os.path.join(os.getcwd(), report_filename)

# JSON 보고서 저장
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

print(f"보고서가 저장되었습니다: {report_filename}")
print(f"저장 경로: {report_path}")

보고서가 저장되었습니다: 20260126_215650_업종_인식률_개선안.json
저장 경로: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\scripts\신승목\20260126_215650_업종_인식률_개선안.json


In [20]:
# JSON 보고서 미리보기
print("" + "=" * 70)
print("JSON 보고서 미리보기")
print("=" * 70)

# 요약 정보 출력
print(f"[요약]")
print(f"  - 보고서 유형: {report['report_type']}")
print(f"  - 대상 파일: {report['target_file']}")
print(f"  - 분석 일시: {report['analysis_date']}")
print(f"  - 전체 테스트: {report['summary']['total_test_cases']}개")
print(f"  - 정확 인식: {report['summary']['correct_detections']}개")
print(f"  - 인식 실패: {report['summary']['failed_detections']}개")
print(f"  - 전체 정확도: {report['summary']['overall_accuracy']}%")
print(f"  - 분석된 그룹 수: {report['groups_analyzed']}개")

# 등급별 정확도
print(f"[등급별 정확도]")
for grade, stats in report['grade_accuracy'].items():
    print(f"  - {grade}: {stats['correct']}/{stats['total']} ({stats['accuracy']}%)")

# 개선 제안 요약
if report['suggestions_by_group']:
    print(f"[그룹별 개선 제안 요약]")
    for group, suggestion in report['suggestions_by_group'].items():
        priority = suggestion.get('priority', 'N/A')
        summary = suggestion.get('summary', 'N/A')[:80] + '...' if len(suggestion.get('summary', '')) > 80 else suggestion.get('summary', 'N/A')
        print(f"  - {group} (우선순위: {priority}): {summary}")

JSON 보고서 미리보기
[요약]
  - 보고서 유형: 업종 인식률 개선사항
  - 대상 파일: src/generation/image_generation/prompt/config/industries.yaml
  - 분석 일시: 2026-01-26 21:56:50
  - 전체 테스트: 494개
  - 정확 인식: 291개
  - 인식 실패: 203개
  - 전체 정확도: 58.9%
  - 분석된 그룹 수: 21개
[등급별 정확도]
  - S등급: 79/106 (74.5%)
  - A등급: 59/82 (72.0%)
  - B등급: 55/92 (59.8%)
  - C등급: 58/114 (50.9%)
  - D등급: 33/90 (36.7%)
  - E등급: 7/10 (70.0%)
[그룹별 개선 제안 요약]
  - S-1 (우선순위: 높음): 현재 실패 케이스는 주로 키워드의 일반성과 문맥 이해 부족에서 기인합니다. S-1 카테고리에 관련된 구체적인 음식 키워드를 추가하고, 일반적인 ...
  - S-2 (우선순위: 중간): S-2 그룹의 업종 인식 실패는 키워드의 다의성 및 문맥 부족으로 인한 혼동에서 비롯되었습니다. 신선한 식재료 관련 키워드를 추가하여 보다 명확...
  - S-3 (우선순위: 중간): S-3 그룹의 키워드 설정을 개선하여 문맥을 더 잘 파악하고, 감정적 판촉을 더 정확히 인식할 수 있도록 해야 합니다.
  - S-4 (우선순위: 중간): S-4 그룹의 정확도를 높이기 위해, 일반 상점과 관련된 키워드를 추가하고 감성적 단어의 우선순위를 조정하는 것이 필요합니다. 이를 통해 유사한...
  - S-5 (우선순위: 높음): 현재 글로벌 요리와 특정 레스토랑 유형에 대한 인식이 부족하여 다양한 요리명을 명확히 구분할 수 있는 키워드 추가와 감지 전략 개선이 필요합니다...
  - S-6 (우선순위: 높음): S-6 (s6_nightlife)에 대한 인식 실패는 주로 특정 키워드의 부재와 문맥적 분석의 부족에서 기인합니다. 이 업종과 직접적으로 관련된.

## 7. 결론

본 분석을 통해 다음을 수행하였습니다:

1. **업종 인식 테스트**: 총 247개 세부 업종당 2개 테스트 입력, 총 494개 테스트 케이스에 대한 인식률 평가
2. **정확도 분석**: 그룹별/등급별 정확도 산출 및 시각화
3. **실패 케이스 분석**: 인식 실패 패턴 및 원인 파악
4. **개선안 도출**: GPT API를 활용한 industries.yaml 개선 방안 제시

### 다음 단계
- 생성된 보고서의 개선안을 참고하여 `industries.yaml` 파일 수정
- 수정 후 재테스트를 통해 인식률 개선 확인